# Tutorial 02 - Fitting Datasets with `curve_fit`

Fitting data to a model or prediction is an extremely common experimental data analysis procedure. This tutorial introduces the function `curve_fit` found in the `scipy` module. Let's look at an example. First `import numpy` and `curve_fit` from `scipy.optimize`. Throw `matplotlib.pylab` in there too.

In [ ]:
import numpy as np
from scipy.optimize import curve_fit
import matplotlib.pylab as plt

Let's create a dataset. We will start by creating a linear array using `linspace`. Any line that starts with a `#` is consider a comment and not part of the code. We will utilize comments when we work on writing good documenation.

In [ ]:
# numpy.linspace with the given arguments
# produce an array of 40 numbers between 0
# and 10, both inclusive
x = np.linspace(0, 10, num = 40)

Next, we will define an array, y, as a function of x. In this case, the function is a sine wave. However, we will also add some "noise" to the data points in order to simulate actual measurements. We will use the `np.random.normal()` function which returns a value from a Gaussian distribution centered around the default value which in this case is zero. Adding the `size=40` flag in the argument makes the function return an array that has 40 elements so that it can be directly added to the y array which itself has 40 elements since it is a function of the 40 element original linspace array x.

In [ ]:
# y is another array which stores 3.45 times
# the sine of (values in x) * 1.334. 
# The random.normal() draws random sample 
# from normal (Gaussian) distribution to make
# them scatter across the base line
y = 3.45 * np.sin(1.334 * x) + (np.random.normal(size = 40))

Let's plot the data so we can see what this looks like. It should look vaguely like a sine function. You can rerun the previous code block to get a new randomized array. The replot to see it.

In [ ]:
# use the keyword argument in quotes, 'o', tells plot to make circles at each point rather than a continous line
plt.plot(x,y,'o')

In order to use `curve_fit`, we have to define a function that we want to fit the data to. Sometimes these functions already exist or are define, but often we need to define our own function. In particular, we need to define a function that has parameters that can very in order to adjust the fit to the data. Let's define a sine function with two parameters, a and b.

In [ ]:
def test(x, a, b):
    return a * np.sin(b * x)

Try testing the `test` function. Put in some values for x, a, and b, and see what you get back. Try putting in the array we made above, x, and then plot the results. You can put the output of the function into a variable by setting it equal to the variable (something like `k=test(x, 3.4, np.pi/2)`

Now, let run `curve_fit` using our test function `test`. `curve_fit` outputs two things: an array of the best fit values for the free parameters in the function (in this case, a and b), and a covariance of the coefficents matrix. The following code block will also plot the `test` function with the returned fit values and compare to the dataset above.

In [ ]:
# curve_fit() function takes the test-function
# x-data and y-data as argument and returns 
# the coefficients a and b in param and
# the estimated covariance of param in param_cov
param, param_cov = curve_fit(test, x, y)
 
print("Sine function coefficients:")
print(param)
print("Covariance of coefficients:")
print(param_cov)
 
# ans stores the new y-data according to 
# the coefficients given by curve-fit() function
ans = (param[0]*(np.sin(param[1]*x)))

plt.plot(x, y, 'o', color ='red', label ="data")
plt.plot(x, ans, '--', color ='blue', label ="optimized data")
plt.legend()
plt.show()

Generally, we can extrat the error in the fit values from the covariance of the coefficient matrix (the second output) by taking the square root of the diagonal elements of the matrix.

In [ ]:
perr = np.sqrt(np.diag(param_cov))
perr

We can then report the fit values with their accompanying error (with some rounding to reduce the significant figures)

In [ ]:
print('The value of a is', np.round(param[0],decimals=4),'+/-', np.round(perr[0],decimals=4))
print('The value of b is', np.round(param[1],decimals=4),'+/-', np.round(perr[1],decimals=4))

Recall that the original unrandomized sine function had a=3.45 and b=1.334. Do these values fall within the error range of the fit? Go back up to the code block that defines y and reduce the effect of the randomization by multiplying `np.random.normal(size=40)` by `0.1` and then rerun all the remaining blocks. How well did the fit do this time?

Next, let's try something a bit more advanced. It is fortunate that `curve_fit` will work even if we do not have equally spaced independant variale data. We can also incorporate measurement error from the data into the fits and use this error to evaluate the goodness of fit using a chi-squared method. First, consider a dataset defined as follows. There is an x-variable, a y-variable, and an error in the y-variable value. We will also plot it using `errorbar` so that the extend of the error range on each datapoint can be visualized.

In [ ]:
x=np.array([8.213,7.402,6.876,5.491,5.196])
y=np.array([3.261,2.52,2.239,1.299,1.175])
y_err=np.array([9.71e-2,5.59e-2,7.08e-2,6.83e-2,8.93e-2])
plt.errorbar(x,y,yerr=y_err,linestyle='none',marker='o',markersize=3,color='red',capsize=2,label='Data')
plt.show()

Let's make a fit to a linear function like y=a+bx. We should define our function as we did before to put into `curve_fit`. Note that this function has more formal documentation style included using the current prefered syntax for Python functions. We'll learn about this style later on this semester.

In [ ]:
def linearFunc(x,intercept,slope):
    """This function defines the function to be fit. In this case a linear
    function.
    
    Parameters
    ----------
    x : independent variable
    slope : slope
    intercept : intercept
    
    Returns
    -------
    y : dependent variable
    """
    y = intercept + slope * x
    return y

Now we can run `curve_fit`. Note that this time we are including the error in y as part of the fit. This provides the fitting routine with a bit more guidance on what would make the best fit parameters for slope and intercept. The flag `absolute_sigma=True` indicates that the value of the error should be treated as an actual error range and not a relative or percentage error. Note, we do not include an error in the x-values. In fact, this is one weakness of `curve_fit` in that it is not easy to incorporate error in the independant variable. There will absolutely be times when there is an error in x to consider, but for these cases we will either have to let it slide, or find a more sophisticated method of fitting.

In [ ]:
param2,param_cov2=curve_fit(linearFunc,x,y,sigma=y_err,absolute_sigma=True)
perr2 = np.sqrt(np.diag(param_cov2))
print('The value of intercept is', np.round(param2[0],decimals=4),'+/-', np.round(perr2[0],decimals=4))
print('The value of slope is', np.round(param2[1],decimals=4),'+/-', np.round(perr2[1],decimals=4))
# Create a graph showing the data.
plt.errorbar(x,y,yerr=y_err,linestyle='none',marker='o',markersize=3,color='red',capsize=2,label='Data')

# Compute a best fit line from the fit intercept and slope.
yfit = param2[0] + param2[1]*x

# Create a graph of the fit to the data. We just use the ordinary plot
# command for this.
plt.plot(x,yfit,label='Fit')

# Display a legend, label the x and y axes and title the graph.
plt.legend()
plt.xlabel('x')
plt.ylabel('y')
plt.show()

Since for this dataset, there is no actual value to compare to, we have to determine goodness of fit in a different way. We will use the reduced chi-squared metric which we will see in Taylor. For a linear fit with N points $\{x_i, y_i \}$ that have uncertainties $\{\delta y_i\}$, we can define $\chi^{2}_{r}$ as

$$
\chi^{2}_{r} = \frac{1}{N-2}\sum^{N}_{i=1}\frac{(y_i-y(x_i))^2}{(\delta y_i)^2}
$$

where $y(x)=a+bx$. For a good fit, $\chi^{2}_{r}$ should be approximately equal to one.

In [ ]:
chisqr = sum((y-linearFunc(x,param2[0],param2[1]))**2/y_err**2)
dof = len(y) - 2
chisqr_red = chisqr/dof
print(f'Reduced chi^2 = {chisqr_red}')

In this case, the fit is pretty good. Try going back and messing with the values in the data array to see how your fits do as well as the value of $\chi^{2}_{r}$. If the fit is not very good, the value of $\chi^{2}_{r}$ becomes much greater than 1. If it is less than 1, this is an indication that the model is overfitting---or, in other words, the amount of error estimated is much too large for this model.

Lastly let's look at fitting a Gaussian to a distribution---again, something that occurs very frequently in data analysis. In Homework 1, you were asked to look at the documenation on `numpy.random.normal`. On that page, the example showed how you could create a distribution from a specific Gaussian distribution and then compare the produced distriution to the actual Gaussian function curve. This time, let's try to fit a Gaussian to the function and see how close it gets to the actually Gaussian from which the distriution was drawn. First create the distribution from a predefined mu and sigma, and then plot the distribution and the Gaussian curve. We will use fewer points in our distribution to make a courser histogram that is less likely to fit closely to the original.

In [ ]:
mu, sigma, samples_num = 0, 0.1, 300 # mean and standard deviation
s = np.random.normal(mu, sigma, samples_num)
count, bins, ignored = plt.hist(s, 30, density=True)
plt.plot(bins, 1/(sigma * np.sqrt(2 * np.pi)) *
               np.exp( - (bins - mu)**2 / (2 * sigma**2) ),
         linewidth=2, color='red')
plt.show()

First we have to define our fitting function

In [ ]:
def gauss_fit(x,mu,sigma):
    """This function defines the Gaussian function to be fit.
    
    Parameters
    ----------
    x : independant variable
    mu : central value of the Gaussian
    sigma : width of the Gaussian
    
    Returns
    -------
    y : Gaussian function defined as 1/(np.sqrt(2*np.pi*sigma**2))*exp**((x-mu)**2/(2*sigma**2))
    """
    a = 1/(np.sqrt(2*np.pi*sigma**2))
    b = (x-mu)**2/(2*sigma**2)
    y = a * np.exp(-b)
    return y

In [ ]:
# plot distribution
count, bins, ignored = plt.hist(s, 30, align='mid',density=True)
bins_centered = (bins[1:] + bins[:-1])/2

#apply curve_fit
param3,param_cov3=curve_fit(gauss_fit,bins_centered,count)
perr3 = np.sqrt(np.diag(param_cov3))
print('The value of mu is', np.round(param3[0],decimals=4),'+/-', np.round(perr3[0],decimals=4))
print('The value of sigma is', np.round(param3[1],decimals=4),'+/-', np.round(perr3[1],decimals=4))

# plot source Gaussian in red
plt.plot(bins_centered, 1/(sigma * np.sqrt(2 * np.pi)) *
               np.exp( - (bins_centered - mu)**2 / (2 * sigma**2) ),
         linewidth=2, color='red',label='source')
# plot fit Gaussian in green
plt.plot(bins_centered, 1/(param3[1] * np.sqrt(2 * np.pi)) *
               np.exp( - (bins_centered - param3[0])**2 / (2 * param3[1]**2) ),
         linewidth=2, color='green',label='fit')
plt.show()

How well does the fit function (green) compare to the original Gaussian from which the distribution was drawn. Go back and play with the number of samples drawn from the distribution and see how that effects the overlap of the two Gaussian curves. Note: to get the fitting right, I had to redefine the bins so that they were defined at their centers rather than their edges.